# Siamese Network for One Shot Learning (Transfer Learning Applied)

### 1. Recap and introduction:
So we saw that our siamese net did not perform quite well and we concluded that the possible reasons could be less data or poor architecture. The model we had was an overfit. We improve the model using transfer learning and we use the model weights of a standard VGG16 model (with weights corresponding to the imagenet dataset). 

### 2. Imports:

In [1]:
import os
import numpy as np
import pickle
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras import applications
from keras import layers
from keras import Model

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 3. Load the VGG16 weights:

In [2]:
model = applications.VGG16(weights = "imagenet", include_top=True, input_shape = (224,224,3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### 4. Preprocess the data for input:

In [3]:
imgs_path = '/home/amula/Desktop/TransLearnPC/NewTest/DataProcessed'
imlist = sorted(os.listdir(imgs_path))
imgs_list = []
for im in imlist:
    img = image.load_img('/home/amula/Desktop/TransLearnPC/NewTest/DataProcessed' + '/' + im, target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis=0)
    x = preprocess_input(x)
    imgs_list.append(x)
imgs_list = np.array(imgs_list)
#print(imgs_list.shape)
imgs_list = np.rollaxis(imgs_list,1,0) #this is the input format required for VGG
imgs_list = imgs_list[0]
#print(imgs_list.shape)

### 5. Lets take the output from the 'fc1' layer: 

In [4]:
intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer("fc1").output)
#features = intermediate_layer_model.predict(imgs_list)
#print(features.shape) #this would answer (600,4096)

### 6. Saving the features to be loaded later:

In [5]:
#file_name = "pickpick.plk"
#fileObject = open(file_name,'ab')
#pickle.dump(features,fileObject)
#fileObject.close()

### 7. Loading the features:

In [6]:
file_name = "/home/amula/Desktop/TransLearnPC/NewTest/pickpick.plk"
fileObject = open(file_name,'rb')
features=pickle.load(fileObject)
features1=pickle.load(fileObject)
fileObject.close()
print(features.shape)
print(features1.shape)
print(np.array_equal(features,features1)) #since the data was quite large we had to save it in two parts of 300 

(300, 4096)
(300, 4096)
False


In [7]:
#Combine the features now
featurescombined = np.zeros([600,4096])
featurescombined[:300,:] = features[:,:]
featurescombined[300:,:] = features1[:,:]

In [8]:
#separating Anchor Positive and Negative
featuresAnc = np.zeros([200,4096])
featuresPos = np.zeros([200,4096])
featuresNeg = np.zeros([200,4096])

for i in range(600):
	if (i+1)%3 == 1:
		featuresAnc[int(i/3),:] = featurescombined[i,:]
	elif (i+1)%3 == 2:
		featuresPos[int(i/3),:] = featurescombined[i,:]
	elif (i+1)%3 == 0:
		featuresNeg[int(i/3),:] = featurescombined[i,:]

In [9]:
#Arranging as Anc-Pos and Anc-Neg
featuresAncPos = np.zeros([200,2*4096+1])
featuresAncNeg = np.zeros([200,2*4096+1])

for i in range(200):
	featuresAncPos[i,:4096] = (featuresAnc[i,:])
	featuresAncPos[i,4096:2*4096] = (featuresPos[i,:])
	featuresAncPos[i,2*4096] = 1
	
	featuresAncNeg[i,:4096] = (featuresAnc[i,:])
	featuresAncNeg[i,4096:2*4096] = (featuresNeg[i,:])
	featuresAncNeg[i,2*4096] = 0
	
featuresMix = np.zeros([400,2*4096+1])
for i in range(400):
	if i<200:
		featuresMix[i,:] = featuresAncPos[i,:]
	else:
		featuresMix[i,:] = featuresAncNeg[i-200,:]

### 8. Shuffle and split:

In [10]:
from sklearn.utils import shuffle
featuresShuffle = shuffle(featuresMix, random_state = 2)

from sklearn.cross_validation import train_test_split
Xytrain,Xytest = train_test_split(featuresShuffle, test_size = 0.2, random_state = 4)

print(Xytrain.shape)
print(Xytest.shape)

X_train = Xytrain[:,:4096]-Xytrain[:,4096:2*4096]
y_train = Xytrain[:,2*4096]

X_test = Xytest[:,:4096]-Xytest[:,4096:2*4096]
y_test = Xytest[:,2*4096]

(320, 8193)
(80, 8193)


/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 9. Finally apply a classifier:

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
clf1 = SVC()
clf1.fit(X_train, y_train)
train_predict1 = clf1.predict(X_train)
test_predict1 = clf1.predict(X_test)
print("Train Accuracy: ", accuracy_score(train_predict1, y_train))
print("Test Accuracy: ", accuracy_score(test_predict1, y_test))

Train Accuracy:  1.0
Test Accuracy:  0.4625


### 10. Summary:
still pretty lame accuracy, hence we conclude that the data should be more which is expected for a computer 
vision problem